In [31]:
import pyranges as pr
import pandas as pd
import os
import glob
import sys
import numpy as np

source_path = os.path.abspath("../../source/")
sys.path.append(source_path)
import utils as ut

In [22]:
chrom = "chr1"
chrom_num = chrom.replace("chr", "")

In [23]:
fpath = "/scratch/indikar_root/indikar1/shared_data/higher_order/reference/gene_table.parquet"

columns = [
    'gene_name',
    'gene_biotype',
    'is_tf',
    'Chromosome',
    'Start',
    'End',
    
]

gdf = pd.read_parquet(fpath, columns=columns)
gdf = gdf[gdf['Chromosome'] == chrom_num]
gdf = pr.PyRanges(gdf)
gdf.head()

/home/cstansbu/miniconda3/envs/higher_order/lib/python3.12/site-packages/pyranges/methods/init.py:45: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  return {k: v for k, v in df.groupby(grpby_key)}


,gene_name,gene_biotype,is_tf,Chromosome,Start,End
0,Gm38212,TEC,False,1,108344806,108347562
1,Gm7449,processed_pseudogene,False,1,6980783,6981446
2,Gm37108,processed_pseudogene,False,1,6986782,6993812
3,Gm37275,processed_pseudogene,False,1,6999982,7000012
4,Gm6679,processed_pseudogene,False,1,108697864,108699733
5,Gm37225,processed_pseudogene,False,1,7068440,7068765
6,Gm29157,lncRNA,False,1,43782074,43782686
7,Gm37489,processed_pseudogene,False,1,7083985,7084144


In [29]:
file_list = glob.glob("/scratch/indikar_root/indikar1/shared_data/population_pore_c/align_table/*.align_table.parquet")

slack = 1000
columns = [
    'read_name',
    'chrom',
    'ref_start',
    'ref_end',
]

res = []

for fpath in file_list:
    file_id = os.path.basename(fpath)
    tmp = pd.read_parquet(fpath, columns=columns)
    tmp.columns = ['read_name', 'Chromosome', 'Start', 'End']
    
    # drop missing mappings
    tmp = tmp[tmp['Chromosome'] == chrom_num]
    tmp = pr.PyRanges(tmp)
    
    tmp = tmp.join(gdf, 
                   slack=slack,
                   suffix='_gene',
                   report_overlap=True)
    
    tmp = tmp.df
    
    tmp['gene_order'] = tmp.groupby('read_name')['gene_name'].transform('nunique')
    tmp = tmp[tmp['gene_order'] > 1]
    tmp['file'] = file_id
    res.append(tmp)
    
    
res = pd.concat(res)
print(f"{res.shape=}")
res = res.sort_values(by='read_name')
res['read_code'] = res['read_name'].astype('category').cat.codes
res.head()

/home/cstansbu/miniconda3/envs/higher_order/lib/python3.12/site-packages/pyranges/methods/init.py:45: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  return {k: v for k, v in df.groupby(grpby_key)}
/home/cstansbu/miniconda3/envs/higher_order/lib/python3.12/site-packages/pyranges/methods/init.py:45: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  return {k: v for k, v in df.groupby(grpby_key)}
/home/cstansbu/miniconda3/envs/higher_order/lib/python3.12/site-packages/pyranges/methods/init.py:45: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass

res.shape=(313647, 12)


,read_name,Chromosome,Start,End,gene_name,gene_biotype,is_tf,Start_gene,End_gene,Overlap,gene_order,file,read_code
3,000215c3-3fdc-4938-a1ff-5f208741fcde,1,57011325,57012324,9130024F11Rik,lncRNA,False,57010570,57089243,2754,3,batch03.GRCm39.align_table.parquet,0
4,000215c3-3fdc-4938-a1ff-5f208741fcde,1,62800674,62800869,Nrp2,protein_coding,False,62742443,62857854,2195,3,batch03.GRCm39.align_table.parquet,0
2,000215c3-3fdc-4938-a1ff-5f208741fcde,1,57011325,57012324,Satb2,protein_coding,True,56833139,57017809,2999,3,batch03.GRCm39.align_table.parquet,0
6,00023d22-f6bc-4012-b148-c983a910ddf9,1,91287488,91288439,Klhl30,protein_coding,False,91278737,91290138,2951,2,batch01.GRCm39.align_table.parquet,1
7,00023d22-f6bc-4012-b148-c983a910ddf9,1,91093842,91093938,Rbm44,protein_coding,False,91072810,91098517,2096,2,batch01.GRCm39.align_table.parquet,1


In [32]:
val = np.ones(len(res))
incidence_matrix = ut.incidence_by_pivot(res, 'read_code', 'gene_name', val)
incidence_matrix.shape

(3510, 83636)

In [33]:
incidence_matrix.head()

read_code,0,1,2,3,4,5,6,7,8,9,...,83626,83627,83628,83629,83630,83631,83632,83633,83634,83635
gene_name,,,,,,,,,,,,,,,,,,,,,
1110002O04Rik,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1600012P17Rik,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1700001G17Rik,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1700003I22Rik,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1700006P03Rik,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
